### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
AY = 'AY 2022-2023'
CO = 'Brain and Behavior'

### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [2]:
# initiate
import numpy as np
import pandas as pd
import re
from datetime import datetime
import datetime
import time
import warnings
warnings.simplefilter("ignore")

# start timer
tic = time.time()

# import dataset
read = pd.read_excel('input.xlsx', engine='openpyxl')

#filter in applicable data only
readfile = read[(read.Status.isin(['Response Type', 0]))]

# subset site survey questions
q_subset = readfile.loc[:,readfile.columns.str.contains('.*Q2.*')]

# create a list of columns
q_cols_subset = list(q_subset.columns)

# calculate the number of one set of questions
set_n = sum(1 for i in q_cols_subset if i[0:2] == (q_cols_subset[0])[0:2])

# create a dataframe for faculty name columns

sequence1 = 0
Faculty_FirstName = []
Faculty_LastName = []

# extract from the first question & repeat set_n times
for k in q_cols_subset:
    sequence1 = sequence1 + 1
    if (sequence1%set_n == 1):            
        q_text_f = str(q_subset[k][0])



        sub1 = "Dr."
        sub2 = "</"

        # getting index of substrings
        idx1 = q_text_f.index(sub1)
        idx2 = q_text_f.index(sub2)

        res = ''
        # getting elements in between
        for idx in range(idx1 + len(sub1) + 1, idx2):
            res = res + q_text_f[idx]

        # result
        Faculty_FullName = res.split()
        Faculty_FirstName.append(str(Faculty_FullName[0]))
        Faculty_LastName.append(str(Faculty_FullName[1]))

    else:
        Faculty_FirstName.append(str(Faculty_FullName[0]))
        Faculty_LastName.append(str(Faculty_FullName[1]))
        continue

df1 = pd.DataFrame(list(zip(Faculty_FirstName, Faculty_LastName)),
       columns =['Faculty_FirstName', 'Faculty_LastName'])
#df1

# create a dataframe for stats

df2 = pd.DataFrame([])
for j in q_cols_subset:

    #counts & response rates
    c = q_subset[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')
    c['pct'] = c.counts / c.counts.sum()

    cc = {}
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']
            pn = c[c.unique_values == i]['pct']
            cc.update(                {
                    'C'+str(i):int(cn), 'R'+str(i):float(pn)
                }            )
            
        else:
            cc.update(                {
                    'C'+str(i):0, 'R'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['A_SA'] = dd.R4 + dd.R5
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    df2 = pd.concat([df2, dd], axis=0, ignore_index=True)
    df2 = df2[['R1', 'R2', 'R3', 'R4', 'R5', 'A_SA', 'Mean', 'N', 'C1', 'C2', 'C3', 'C4', 'C5']]

#df2

# create a dataframe for question by extracting text

sequence2=0
q0 = []
for l in q_cols_subset:
    if (sequence2 < set_n):        
        sequence2 = sequence2 + 1
        q_text_q = str(q_subset[l][0])  
        q0.append(q_text_q.split("/> - ", 1)[1])
    else:
        break

q1 = pd.DataFrame(q0)

count = 0
df3 = pd.DataFrame([])
while (count < (len(q_cols_subset)/set_n)):   
    count = count + 1
    df3 = pd.concat([df3, q1], axis=0, ignore_index=True)

df3.columns =['Question']
#df3

# 5 concat three dataframes
df4 = pd.concat([df3, df1, df2], axis=1)
#df4

# delete qualitative questions
df5 = df4[df4['N'] > 0]

# populate columns for AY, Class, Block
df5['AcademicYear'] = AY
df5['Course'] = CO
df5['NetID'] = ''
df5['GWID'] = ''
df5['Department'] = ''
df5['Course Code'] = ''
df5[', '] = ', '
df5['Faculty'] = df5.Faculty_LastName +', ' + df5.Faculty_FirstName
df5 = df5[['NetID', 'GWID', 'Department', 'Faculty', 'Faculty_FirstName', 'Faculty_LastName',
           'AcademicYear', 'Course', 'Course Code', 'Question',
           'R1', 'R2', 'R3', 'R4', 'R5', 'A_SA', 'Mean', 'N', 'C1', 'C2', 'C3', 'C4', 'C5']]
#df5

# export
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 3.04 s.
